## Cargar Ficheros Multilingual-USAS
#Spanish Semantic MWE Lexicon

multipalabras  
https://github.com/UCREL/Multilingual-USAS/blob/master/Spanish/mwes_es.usas



In [1]:
#Creamos un RDD a partir de una fuente de almacenamiento utilizando sc.textFile del SparkContext
# Aplicamos las transformaciones (map: aplica una función a cada elemento de la colección)  para limpiar los datos y preparar la tabla
# Importar tipos de datos

from pyspark.sql.types import *
import re #This module provides regular expression 

p = re.compile( '(_adj|_adv|_art|_part|_conj|_intj|_noun|num_|_prep|_pron|_verb|_pnoun|_port|_abbr|_sys|_punc|_fw)') #Pattern for cleaning text 
multi_word_RDD = sc.textFile('mwes_es.usas.txt', 8)
multi_word_filterRDD= multi_word_RDD.filter(lambda l: not l.startswith("#") and len(l) > 1).map(lambda elemento: p.sub( '', elemento)).map(lambda x: x.encode('ascii','ignore').split('\t'))

In [2]:
# Creamos el esquema

fields_mw = [StructField("TOKEN", StringType(), True),StructField("SEMTAG", StringType(), True)]
schema_mw = StructType(fields_mw)

# Crear el dataframe (infiriendo el esquema)
dfmulti_words = spark.createDataFrame(multi_word_filterRDD,schema_mw )

# Registrar el DataFrame como tabla.
dfmulti_words.createOrReplaceTempView("multi_words")


# Ejecutar SQL
multi_word_query = spark.sql("SELECT SEMTAG, TOKEN FROM multi_words WHERE TOKEN = 'A Arnoia' ")

In [3]:
multi_word_query.show()

+------+--------+
|SEMTAG|   TOKEN|
+------+--------+
|    Z2|A Arnoia|
+------+--------+



In [4]:
#if multi_word_query.collect()[0].TOKEN == "A Arnoia":
#    print multi_word_query.collect()[0].SEMTAG , multi_word_query.collect()[0].TOKEN


In [5]:
# Guardamos Fichero en formato parquet
spark.sql("SELECT * FROM multi_words").write.save("multi_words.parquet", mode='overwrite')

In [9]:
# Para cargarlo desde Parquet y hacer querys
df = spark.read.load("multi_words.parquet")

# Registrar el DataFrame como tabla.
df.createOrReplaceTempView("multi_words")

palabra = "El Port de la Selva"
query = 'SELECT * FROM dbsnp where rs_id="{}"'

# Ejecutar SQL
multi_word_query = spark.sql('SELECT *  FROM multi_words WHERE TOKEN =="{}"'.format(palabra))
multi_word_query.show()

+-------------------+------+
|              TOKEN|SEMTAG|
+-------------------+------+
|El Port de la Selva|    Z2|
+-------------------+------+



In [ ]:
"SELECT col1 from table where col2>{0} limit {1}".format(var2,q25)

shepherd = "Mary"
>>> string_in_string = "Shepherd {} is on duty.".format(shepherd)